In [1]:
import pandas as pd
import random
import numpy as np
from konlpy.tag import Twitter
from itertools import chain
import collections
import tensorflow as tf

C:\Users\user\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
from rnn_model.attention_lstm import bilstm_with_attention

In [4]:
bilstm_with_attention

rnn_model.attention_lstm.bilstm_with_attention

In [6]:
total_balanced_data = pd.read_csv("total_balance_final.csv")

In [7]:
def random_row_sampling(df,n):
    return df.ix[np.random.random_integers(0,len(df),n)]

In [8]:
sampled_data = random_row_sampling(total_balanced_data,50000)

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: This function is deprecated. Please call randint(0, 3000000 + 1) instead
  


In [9]:
sampled_data['labels_text'].value_counts() ## 균등하게 분배됨

__label__25    1773
__label__21    1768
__label__3     1719
__label__7     1715
__label__2     1710
__label__28    1707
__label__14    1694
__label__4     1692
__label__9     1687
__label__12    1684
__label__10    1681
__label__8     1680
__label__20    1671
__label__22    1665
__label__23    1661
__label__0     1648
__label__13    1647
__label__19    1645
__label__17    1643
__label__11    1640
__label__24    1637
__label__27    1634
__label__15    1633
__label__29    1633
__label__6     1632
__label__26    1632
__label__16    1631
__label__5     1629
__label__18    1607
__label__1     1602
Name: labels_text, dtype: int64

In [10]:
sampled_data.shape

(50000, 3)

In [11]:
sampled_data.head()

,Unnamed: 0,labels_text,tweet
1009115,2337412,__label__10,하여튼 여러모로 좀 아쉽 이제 첫 년을 보냈고 리그가 시작하면서 거치는 과도기라고 ...
142954,234995,__label__1,최고의 리더 생일 축하해요
1280787,2756311,__label__12,의원님 지금 잠이 오지 않아요 인터뷰 공무원 보도한 반드시 책임 물어 주세요 절 대...
1875221,3871130,__label__18,강진 안녕하세요 목구멍 무리하지 마세요
2649579,6140365,__label__26,비양 이이 이이ㅣ이ㅣ이링 내가 좋아하는 몇 없는 내 동생 너무 착해 ㅜㅜㅜㅜㅜ


In [12]:
train_data = sampled_data.iloc[:40000,:]
test_data = sampled_data.iloc[40000:,:]

In [13]:
def tokenize(doc):  ## 리스트형테의 데이터 안에서 리뷰부분을 토크나이징해서 다시 리스트로 저장

    return ['/'.join(t) for t in pos_tagger.pos(doc, norm=True, stem=True)]

In [14]:
pos_tagger = Twitter()

C:\Users\user\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [15]:
train_data = train_data.reset_index().drop(['index','Unnamed: 0'],axis=1)
test_data = test_data.reset_index().drop(['index','Unnamed: 0'],axis=1)

In [16]:
train_tokens = [tokenize(row) for row in train_data['tweet']]

In [17]:
train_labels = [int(row[9:]) for row in train_data['labels_text']]

In [18]:
test_tokens = [tokenize(row) for row in test_data['tweet']]
test_labels = [int(row[9:]) for row in test_data['labels_text']]

In [19]:
test_labels

[22,
 1,
 9,
 8,
 9,
 13,
 11,
 28,
 1,
 9,
 15,
 13,
 21,
 16,
 29,
 14,
 7,
 13,
 13,
 10,
 3,
 25,
 11,
 9,
 9,
 24,
 22,
 8,
 10,
 17,
 7,
 25,
 20,
 17,
 24,
 27,
 16,
 27,
 4,
 15,
 5,
 4,
 5,
 23,
 9,
 18,
 28,
 9,
 15,
 26,
 4,
 11,
 15,
 28,
 4,
 16,
 13,
 22,
 4,
 26,
 22,
 17,
 3,
 2,
 11,
 15,
 2,
 23,
 24,
 24,
 28,
 9,
 7,
 15,
 26,
 17,
 28,
 2,
 3,
 9,
 28,
 13,
 27,
 16,
 14,
 26,
 11,
 7,
 28,
 13,
 16,
 2,
 14,
 15,
 10,
 12,
 0,
 24,
 18,
 16,
 2,
 6,
 9,
 1,
 14,
 7,
 17,
 25,
 24,
 6,
 5,
 5,
 6,
 17,
 26,
 28,
 26,
 20,
 11,
 3,
 16,
 14,
 5,
 18,
 17,
 11,
 8,
 13,
 1,
 16,
 25,
 23,
 15,
 14,
 28,
 24,
 19,
 20,
 25,
 26,
 29,
 25,
 6,
 13,
 20,
 27,
 23,
 19,
 12,
 26,
 20,
 11,
 28,
 29,
 13,
 0,
 17,
 16,
 24,
 21,
 23,
 24,
 3,
 26,
 19,
 7,
 28,
 0,
 10,
 13,
 25,
 29,
 13,
 18,
 6,
 4,
 2,
 8,
 29,
 3,
 18,
 2,
 13,
 22,
 18,
 5,
 2,
 26,
 15,
 13,
 26,
 23,
 10,
 11,
 10,
 12,
 0,
 2,
 18,
 21,
 18,
 13,
 23,
 6,
 7,
 4,
 27,
 29,
 24,
 4,
 18,
 20,
 22,

In [20]:
total_tokens = train_tokens + test_tokens
unlist_tokens = list(chain.from_iterable(total_tokens))
len(total_tokens)

50000

In [21]:
words = list() ## word index를 주기 위해서 dictionary를 만든다 -> lookup table사용
for word in unlist_tokens:
    words.append(word)
word_counter = collections.Counter(words).most_common()
word_dict = dict()
word_dict["<pad>"] = 0
word_dict["<unk>"] = 1
word_dict["<eos>"] = 2
for word,_ in word_counter:
    word_dict[word] = len(word_dict)

In [22]:
x = list(map(lambda d: list(map(lambda w: word_dict.get(w, word_dict["<unk>"]), d)), train_tokens))
x = list(map(lambda d: d[:50], x))
x = list(map(lambda d: d + (50- len(d)) * [word_dict["<pad>"]], x))
y = train_labels
train_x = x
train_y = y

In [23]:
x = list(map(lambda d: list(map(lambda w: word_dict.get(w, word_dict["<unk>"]), d)), test_tokens)) ## 딕셔너리의 인덱스들로 바꿔준다.
x = list(map(lambda d: d[:50], x))  ## 최대길이 50까지만 가져옴.
x = list(map(lambda d: d + (50- len(d)) * [word_dict["<pad>"]], x)) ## 50 전에는 padding을 준다.
y = test_labels
test_x = x
test_y = y

In [24]:
def batch_iter(inputs, outputs, batch_size, num_epochs):
    inputs = np.array(inputs)
    outputs = np.array(outputs)

    num_batches_per_epoch = (len(inputs) - 1) // batch_size + 1
    for epoch in range(num_epochs):
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, len(inputs))
            yield inputs[start_index:end_index], outputs[start_index:end_index]

In [25]:
NUM_CLASS = 30 
BATCH_SIZE = 64
NUM_EPOCHS = 1    ## epoch 1 로 주고 한번 돌려보자
WORD_MAX_LEN = 50 ## 최대토큰길이 50
vocab_size = len(word_dict)

In [26]:
num_batches_per_epoch = (len(train_x) - 1) // BATCH_SIZE + 1
print(num_batches_per_epoch)

625


In [27]:
vocab_size

37093

In [28]:
model = bilstm_with_attention(vocab_size, 50, 30)

Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').


In [29]:
train_batches = batch_iter(train_x, train_y, BATCH_SIZE, NUM_EPOCHS)

In [30]:
list(train_batches)

[(array([[5295, 3715, 3716, ...,    0,    0,    0],
         [ 182,   25, 2302, ...,    0,    0,    0],
         [5297,   93,  348, ...,    0,    0,    0],
         ...,
         [1084,  609,  633, ...,    0,    0,    0],
         [  84,  337,  721, ...,    0,    0,    0],
         [ 185,   86,  172, ...,    0,    0,    0]]),
  array([10,  1, 12, 18, 26, 20,  5,  2,  5, 11,  9,  4, 28,  6,  0,  0, 23,
         19,  4,  1, 12, 26, 10, 21, 12,  1, 20, 23, 18, 27,  4, 10, 23,  9,
          4, 15,  1, 25, 25, 13, 13,  2,  3, 17,  3,  7, 10,  5, 25, 17, 21,
          5, 12, 14,  1, 18, 27,  3, 16, 27, 12,  0, 25, 18])),
 (array([[ 4561,    50,   444, ...,     0,     0,     0],
         [    3,  5025,   148, ...,     0,     0,     0],
         [  431,   499,   488, ...,     0,     0,     0],
         ...,
         [10050,   104, 10051, ...,     0,     0,     0],
         [14513,  2353,   857, ...,     0,     0,     0],
         [  155,  7877,   292, ...,     0,     0,     0]]),
  array([28, 

In [31]:
num_batches_per_epoch

625

In [32]:
with tf.Session() as sess:
    
    model = bilstm_with_attention(vocab_size, 50, 30)  ## vocab_size, max_length, class
    init = tf.global_variables_initializer()
    sess.run(init) 

    saver = tf.train.Saver(tf.global_variables())
    
    train_batches = batch_iter(train_x, train_y, BATCH_SIZE, NUM_EPOCHS)
    num_batches_per_epoch = (len(train_x) - 1) // BATCH_SIZE + 1  
    max_accuracy = 0 

    for x_batch, y_batch in train_batches:
        train_feed_dict = {
            model.x: x_batch,
            model.y: y_batch,
            model.is_training: True
        }

        _, step, loss = sess.run([model.optimizer, model.global_step, model.loss], feed_dict=train_feed_dict)

        if step % 100 == 0:
            print("step {0}: loss = {1}".format(step, loss))

        if (step % 1000 == 0) or (step == num_batches_per_epoch ) : 
            # Test accuracy with validation data for each epoch.
            valid_batches = batch_iter(test_x, test_y, BATCH_SIZE, 1) ## test 단에서는 num_epoch 1로 준다
            sum_accuracy, cnt = 0, 0

            for valid_x_batch, valid_y_batch in valid_batches:
                valid_feed_dict = {
                    model.x: valid_x_batch,
                    model.y: valid_y_batch,
                    model.is_training: False
                }

                accuracy = sess.run(model.accuracy, feed_dict=valid_feed_dict)
                sum_accuracy += accuracy
                cnt += 1
            valid_accuracy = sum_accuracy / cnt

            print("\nValidation Accuracy = {1}\n".format(step // num_batches_per_epoch, sum_accuracy / cnt))

            # Save model
            if valid_accuracy > max_accuracy:
                max_accuracy = valid_accuracy
                saver.save(sess, "{0}/{1}.ckpt".format("BILSTM_attention", "BILSTM_attention"), global_step=step)
                print("Model is saved.\n")

step 100: loss = 3.3966801166534424
step 200: loss = 3.4237635135650635
step 300: loss = 3.4134695529937744
step 400: loss = 3.4072794914245605
step 500: loss = 3.3896493911743164
step 600: loss = 3.4091274738311768

Validation Accuracy = 0.03552945859872612

Model is saved.

